# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [1]:
import pandas as pd
import numpy as np
import subprocess
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

RANDOM_STATE = 42

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [3]:
def load_parquet_file(filename, base_url="https://storage.yandexcloud.net/mle-data/ym/"):
    """
    Универсальная функция для загрузки parquet файлов.
    
    Args:
        filename (str): Имя файла для загрузки (например, "tracks.parquet")
        base_url (str): Базовый URL для скачивания файлов
    
    Returns:
        pd.DataFrame или None: Загруженные данные или None при ошибке
    """
    # Формируем полный URL
    file_url = f"{base_url}{filename}"
    
    try:
        data = pd.read_parquet(filename)
        print(f"{filename} загружен локально")
        return data
        
    except FileNotFoundError:
        print(f"{filename} не найден, скачиваю из {file_url}...")
        try:
            subprocess.run(['wget', file_url], check=True)
            data = pd.read_parquet(filename)
            print(f"{filename} успешно скачан и загружен")
            return data
        except subprocess.CalledProcessError:
            print(f"Ошибка при скачивании {filename}")
            return None
        except Exception as e:
            print(f"Ошибка при чтении скачанного {filename}: {e}")
            return None
    except Exception as e:
        print(f"Ошибка при чтении {filename}: {e}")
        return None

In [4]:
# Использование функции
tracks = load_parquet_file("tracks.parquet")
catalog_names = load_parquet_file("catalog_names.parquet")
interactions = load_parquet_file("interactions.parquet")

tracks.parquet загружен локально
catalog_names.parquet загружен локально
interactions.parquet загружен локально


In [5]:
# Проверяем загрузку
if all(x is not None for x in [tracks, catalog_names, interactions]):
    print("✓ Все файлы успешно загружены!")
    print(f"Tracks: {tracks.shape}")
    print(f"Catalog names: {catalog_names.shape}")
    print(f"Interactions: {interactions.shape}")
else:
    print("⚠ Не все файлы удалось загрузить")

✓ Все файлы успешно загружены!
Tracks: (1000000, 4)
Catalog names: (1812471, 3)
Interactions: (222629898, 4)


# Обзор данных

tracks.parquet - данные о треках:  
 - track_id — идентификатор музыкального трека;  
 - albums —  список идентификаторов альбомов, содержащих трек;  
 - artists — список идентификаторов исполнителей трека;  
 - genres — список идентификаторов жанров, к которым принадлежит трек.

In [6]:
tracks.head()

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]


In [7]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


In [8]:
is_unique = tracks['track_id'].nunique() == len(tracks)
unique_count = tracks['track_id'].nunique()
total_count = len(tracks)

print(f" Уникальность значений:")
print("=" * 40)
print(f"   • Всего записей: {total_count:,}")
print(f"   • Уникальных track_id: {unique_count:,}")
print(f"   • Все track_id уникальны: {'ДА' if is_unique else 'НЕТ'}")
if not is_unique:
    duplicates = total_count - unique_count
    print(f"   • Дубликатов найдено: {duplicates:,}")

 Уникальность значений:
   • Всего записей: 1,000,000
   • Уникальных track_id: 1,000,000
   • Все track_id уникальны: ДА


In [9]:
tracks[tracks["albums"].str.len() == 0]

,track_id,albums,artists,genres
310821,20200372,[],[],[]
310826,20200380,[],[],[]
312469,20305116,[],[],[]
312474,20305121,[],[],[]
320353,20756854,[],[],[]
326588,21196099,[],[],[]
326592,21196103,[],[],[]
326594,21196105,[],[],[]
326596,21196107,[],[],[]
326598,21196109,[],[],[]


есть небольшое кол-во пустых списков идентификаторов альбомов (с пустыми для них идентификаторами исполнителей трека и жанров)

In [10]:
def check_empty_lists(df, column_name, entity_name=None):
    """
    Проверяет колонку с списками на пустые значения
    
    Parameters:
    df - DataFrame для анализа
    column_name - название колонки со списками
    entity_name - человекочитаемое название сущности (опционально)
    """
    
    # Подсчитываем пустые списки
    empty_mask = df[column_name].str.len() == 0
    count_empty = empty_mask.sum()
    total_count = len(df)
    
    print(f"Записей с пустыми {entity_name}: {count_empty:,}")
    print(f"Процент от общего числа: {count_empty/total_count*100:.4f}%")
    print(f"Всего записей: {total_count:,}")
    print(f"Записей с {entity_name}: {total_count - count_empty:,}")
    print("=" * 45)
    
    if count_empty > 0:
        # Получаем примеры ID с пустыми списками
        if 'track_id' in df.columns:
            sample_ids = df.loc[empty_mask, 'track_id'].head(5).tolist()
            print(f"\n Примеры track_id с пустыми {entity_name}:")
            for track_id in sample_ids:
                print(f"   • {track_id}")
        else:
            # Если нет track_id, показываем индексы
            sample_indices = df[empty_mask].index[:5].tolist()
            print(f"\n Примеры индексов с пустыми {entity_name}:")
            for idx in sample_indices:
                print(f"   • Индекс: {idx}")
    
    print("=" * 45)
    return empty_mask

In [11]:
def remove_empty_lists_column(df, column_name, entity_name=None):
    """
    Удаляет записи с пустыми списками в указанном столбце
    
    Parameters:
    df - DataFrame для обработки
    column_name - название столбца со списками
    entity_name - человекочитаемое название сущности
    """
    
    if entity_name is None:
        entity_name = column_name
    
    print(f"УДАЛЕНИЕ ЗАПИСЕЙ БЕЗ {entity_name.upper()}")
    print("=" * 45)
    
    # Сохраняем исходное количество
    initial_count = len(df)
    
    # Находим track_id для удаления
    track_id_to_delete = df[df[column_name].str.len() == 0]["track_id"]
    
    # Удаляем записи
    df_cleaned = df[~df["track_id"].isin(track_id_to_delete)]
    final_count = len(df_cleaned)
    
    print(f"Удалено треков: {initial_count - final_count:,}")
    print(f"Осталось треков: {final_count:,}")
    print("=" * 45)
    
    return df_cleaned, track_id_to_delete.tolist()

In [12]:
albums_result = check_empty_lists(tracks, 'albums', 'альбомами')

Записей с пустыми альбомами: 18
Процент от общего числа: 0.0018%
Всего записей: 1,000,000
Записей с альбомами: 999,982

 Примеры track_id с пустыми альбомами:
   • 20200372
   • 20200380
   • 20305116
   • 20305121
   • 20756854


In [13]:
# Удаляем треки без альбомов
tracks, deleted_albums = remove_empty_lists_column(tracks, "albums", "альбомов")

УДАЛЕНИЕ ЗАПИСЕЙ БЕЗ АЛЬБОМОВ
Удалено треков: 18
Осталось треков: 999,982


In [14]:
artists_result = check_empty_lists(tracks, 'artists', 'артистами')

Записей с пустыми артистами: 15,351
Процент от общего числа: 1.5351%
Всего записей: 999,982
Записей с артистами: 984,631

 Примеры track_id с пустыми артистами:
   • 3599314
   • 3599591
   • 4790215
   • 10063296
   • 12122918


In [15]:
# Удаляем треки без артистов
tracks, deleted_artists = remove_empty_lists_column(tracks, "artists", "артистов")

УДАЛЕНИЕ ЗАПИСЕЙ БЕЗ АРТИСТОВ
Удалено треков: 15,351
Осталось треков: 984,631


In [16]:
genres_result = check_empty_lists(tracks, 'genres', 'жанрами')

Записей с пустыми жанрами: 3,654
Процент от общего числа: 0.3711%
Всего записей: 984,631
Записей с жанрами: 980,977

 Примеры track_id с пустыми жанрами:
   • 2520
   • 16776
   • 16801
   • 23752
   • 38012


In [17]:
tracks, deleted_genres = remove_empty_lists_column(tracks, "genres", "жанров")

УДАЛЕНИЕ ЗАПИСЕЙ БЕЗ ЖАНРОВ
Удалено треков: 3,654
Осталось треков: 980,977


In [18]:
# Объединяем все списки удаленных треков
track_id_deleted = deleted_albums + deleted_artists + deleted_genres

print(" СТАТИСТИКА УДАЛЕННЫХ ТРЕКОВ")
print("=" * 40)

# Подсчитываем общую статистику
total_deleted = len(track_id_deleted)
unique_deleted = len(set(track_id_deleted))

print(f"Всего удалено записей: {total_deleted:,}")
print(f"Уникальных track_id: {unique_deleted:,}")

# Статистика по исходным данным
initial_count = unique_count  # Замените на ваше исходное количество
remaining_count = unique_count - unique_deleted

print(f"\n ОБЩАЯ СТАТИСТИКА:")
print("=" * 40)
print(f"Исходное количество треков: {unique_count:,}")
print(f"Удалено уникальных треков: {unique_deleted:,}")
print(f"Осталось треков: {remaining_count:,}")
print(f"Процент удаленных: {unique_deleted*100/initial_count:.2f}%")

 СТАТИСТИКА УДАЛЕННЫХ ТРЕКОВ
Всего удалено записей: 19,023
Уникальных track_id: 19,023

 ОБЩАЯ СТАТИСТИКА:
Исходное количество треков: 1,000,000
Удалено уникальных треков: 19,023
Осталось треков: 980,977
Процент удаленных: 1.90%


In [19]:
# Сохраняем список в CSV
deleted_df = pd.DataFrame({'deleted_track_id': track_id_deleted})
deleted_df.to_csv('deleted_tracks.csv', index=False)

print(f"Список из {len(track_id_deleted):,} track_id сохранен в 'deleted_tracks.csv'")

Список из 19,023 track_id сохранен в 'deleted_tracks.csv'


In [20]:
# Преобразуем track_id из int64 в int32
tracks['track_id'] = tracks['track_id'].astype('int32')

# Проверим результат
print(f"Новый тип track_id: {tracks['track_id'].dtype}")
print(f"Память после преобразования:")
tracks.info()

Новый тип track_id: int32
Память после преобразования:
<class 'pandas.core.frame.DataFrame'>
Index: 980977 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   track_id  980977 non-null  int32 
 1   albums    980977 non-null  object
 2   artists   980977 non-null  object
 3   genres    980977 non-null  object
dtypes: int32(1), object(3)
memory usage: 33.7+ MB


In [21]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980977 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   track_id  980977 non-null  int32 
 1   albums    980977 non-null  object
 2   artists   980977 non-null  object
 3   genres    980977 non-null  object
dtypes: int32(1), object(3)
memory usage: 33.7+ MB


_______________________________________________

catalog_names.parquet - имена артистов, названия альбомов, треков и жанров:  
 - id — идентификатор одной из каталожных единиц (трека, альбома, исполнителя, жанра);  
 - type — тип идентификатора;  
 - name — имя (название) каталожной единицы.

In [22]:
catalog_names.head()

,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts
2,13,album,Lonesome Crow
3,17,album,Graffiti Soul
4,26,album,Blues Six Pack


In [23]:
catalog_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB


In [24]:
catalog_names["type"].value_counts()
# в каталоге больше всего треков, затем альбомы, артисты и жанры 

type
track     1000000
album      658724
artist     153581
genre         166
Name: count, dtype: int64

In [25]:
catalog_name_id_dubl = len(catalog_names) - catalog_names["id"].nunique()

print(f"Исходное количество `id`: {len(catalog_names):,}")
print(f"Уникальных `id`: {catalog_names['id'].nunique():,}")
print(f"Дубликаты: {catalog_name_id_dubl:,}")

Исходное количество `id`: 1,812,471
Уникальных `id`: 1,776,697
Дубликаты: 35,774


In [26]:
catalog_names[catalog_names['id'].duplicated()]

,id,type,name
658728,12,artist,Phil Everly
658735,34,artist,Miles Davis
658736,36,artist,Baker
658741,87,artist,Michael Schenker
658744,93,artist,Femi Kuti
...,...,...,...
1141043,21314698,track,No Good For You
1141108,21320653,track,Oh!
1141114,21321251,track,Lady in Red
1141142,21327491,track,Waiting Here


In [27]:
catalog_names[catalog_names["id"] == 87]

,id,type,name
25,87,album,Chronicle: 20 Greatest Hits
658741,87,artist,Michael Schenker
812392,87,genre,religion


Идентификаторы `id` для разных типов могут повторяться.

In [28]:
catalog_name_name_dubl = len(catalog_names) - catalog_names["name"].nunique()

print(f"Исходное количество `name`: {len(catalog_names):,}")
print(f"Уникальных `name`: {catalog_names['name'].nunique():,}")
print(f"Дубликаты: {catalog_name_name_dubl:,}")

Исходное количество `name`: 1,812,471
Уникальных `name`: 945,118
Дубликаты: 867,353


есть дубликаты по имени (названию) каталожной единицы. - 867353 из 1812471

In [29]:
catalog_names_t_n_dubl = catalog_names[catalog_names['name'].duplicated()]
catalog_names_t_n_dubl

,id,type,name
10,43,album,Blues Six Pack
11,45,album,Blues Six Pack
13,49,album,Graffiti Soul
28,94,album,Jazz Six Pack
29,95,album,Blues Six Pack
...,...,...,...
1812466,101478482,track,На лицо
1812467,101490148,track,Без капли мысли
1812468,101493057,track,SKITTLES
1812469,101495927,track,Москва


In [30]:
catalog_names[catalog_names["name"] == "Blues Six Pack"]

,id,type,name
4,26,album,Blues Six Pack
10,43,album,Blues Six Pack
11,45,album,Blues Six Pack
29,95,album,Blues Six Pack
34519,91414,album,Blues Six Pack


под разными идентификаторами в каталоге имеются одни и те же альбомы

In [31]:
# удаляем такие дубли оставив первый
catalog_names_clean = catalog_names.drop_duplicates(subset=["type","name"], keep="first")

print(f"\n ОБЩАЯ СТАТИСТИКА:")
print("=" * 40)
print(f"Исходное количество записей: {len(catalog_names):,}")
print(f"Удалено дублированных записей: {len(catalog_names) - len(catalog_names_clean):,}")
print(f"Осталось записей: {len(catalog_names_clean):,}")
print(f"Процент оставшихся записей: {len(catalog_names_clean)*100/len(catalog_names):.2f}%")


 ОБЩАЯ СТАТИСТИКА:
Исходное количество записей: 1,812,471
Удалено дублированных записей: 689,769
Осталось записей: 1,122,702
Процент оставшихся записей: 61.94%


In [32]:
catalog_names_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1122702 entries, 0 to 1812468
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1122702 non-null  int64 
 1   type    1122702 non-null  object
 2   name    1122702 non-null  object
dtypes: int64(1), object(2)
memory usage: 34.3+ MB


In [33]:
# уберем из данных треки из списка track_id_deleted
catalog_names_clean_tracks = catalog_names_clean[catalog_names_clean["type"]=="track"]
catalog_names_clean_others = catalog_names_clean[catalog_names_clean["type"]!="track"]
catalog_names_clean_tracks = catalog_names_clean_tracks[~catalog_names_clean_tracks["id"].isin(track_id_deleted)]
catalog_names_clean = pd.concat([catalog_names_clean_others,catalog_names_clean_tracks],ignore_index=True)

print(f"\n ИТОГОАЯ СТАТИСТИКА:")
print("=" * 40)
print(f"Исходное количество записей: {len(catalog_names):,}")
print(f"Удалено дублированных записей: {len(catalog_names) - len(catalog_names_clean):,}")
print(f"Удалено треков: {len(track_id_deleted):,}")
print(f"Осталось записей: {len(catalog_names_clean):,}")
print(f"Процент оставшихся записей: {len(catalog_names_clean)*100/len(catalog_names):.2f}%")


 ИТОГОАЯ СТАТИСТИКА:
Исходное количество записей: 1,812,471
Удалено дублированных записей: 706,682
Удалено треков: 19,023
Осталось записей: 1,105,789
Процент оставшихся записей: 61.01%


In [34]:
catalog_names_clean.sort_values(by = "name")
# в именах (названиях) каталожных единиц присутсвуют эмодзи, символы ASCII, либо только знаки препинания.
# фильтровать такие значения тяжело, пока оставим как есть, но будем иметь ввиду

,id,type,name
575388,8865715,artist,\tDrake
670765,1710520,track,!
102514,2081150,album,!
567666,7709071,artist,!
32780,169465,album,! (Знак оклику)
...,...,...,...
390658,14512698,album,💗 Pop y Amor 💗
389309,14404588,album,📚 Música Para Leer 📚
386736,14157216,album,🔥 Regeton Viejito Vol.2 🔥
389388,14405050,album,🚗 Para que te sientas en Camioneta Roja 🚗


In [35]:
import re
import pandas as pd

def clean_text(text):
    if pd.isna(text):
        return text
    
    # Удаляем эмодзи и специальные символы
    text = re.sub(r'[^\w\sа-яА-Яa-zA-Z0-9]', ' ', str(text))
    
    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# Применяем функцию к столбцу
catalog_names_clean2 = catalog_names_clean.copy()
catalog_names_clean2['name_clean'] = catalog_names_clean2['name'].apply(clean_text)

In [36]:
catalog_names_clean2.sort_values(by = "name")

,id,type,name,name_clean
575388,8865715,artist,\tDrake,Drake
670765,1710520,track,!,
102514,2081150,album,!,
567666,7709071,artist,!,
32780,169465,album,! (Знак оклику),Знак оклику
...,...,...,...,...
390658,14512698,album,💗 Pop y Amor 💗,Pop y Amor
389309,14404588,album,📚 Música Para Leer 📚,Música Para Leer
386736,14157216,album,🔥 Regeton Viejito Vol.2 🔥,Regeton Viejito Vol 2
389388,14405050,album,🚗 Para que te sientas en Camioneta Roja 🚗,Para que te sientas en Camioneta Roja


In [37]:
# Находим записи, которые после очистки стали пустыми или слишком короткими
empty_names = catalog_names_clean2[
    (catalog_names_clean2['name_clean'].str.len() < 2) | 
    (catalog_names_clean2['name_clean'].isna())
]

print(f"Записей с невалидными названиями: {len(empty_names)}")
print(f"Всего записей: {len(catalog_names_clean2)}")
print(f"Процент записей к удалению: {len(empty_names)*100/len(catalog_names_clean2):.2f}%")

Записей с невалидными названиями: 384
Всего записей: 1105789
Процент записей к удалению: 0.03%


In [39]:
# Удаляем из основного DataFrame записи с индексами из empty_names
catalog_names_clean2 = catalog_names_clean2.drop(empty_names.index)

print(f"Удалено записей: {len(empty_names)}")
print(f"Осталось записей: {len(catalog_names_clean2)}")

Удалено записей: 384
Осталось записей: 1105405


In [40]:
catalog_names_clean2.sort_values(by = "name")

,id,type,name,name_clean
575388,8865715,artist,\tDrake,Drake
32780,169465,album,! (Знак оклику),Знак оклику
670763,1710515,track,! (С.К.А.Й. & Пікардійська Терція),С К А Й Пікардійська Терція
783780,19775700,track,"! Cat Meow Meow, #1 You Are My Sunshine (feat....",Cat Meow Meow 1 You Are My Sunshine feat Fun K...
107427,2228200,album,"! Cat Meow Meow, #1 You Are My Sunshine (feat....",Cat Meow Meow 1 You Are My Sunshine feat Fun K...
...,...,...,...,...
390658,14512698,album,💗 Pop y Amor 💗,Pop y Amor
389309,14404588,album,📚 Música Para Leer 📚,Música Para Leer
386736,14157216,album,🔥 Regeton Viejito Vol.2 🔥,Regeton Viejito Vol 2
389388,14405050,album,🚗 Para que te sientas en Camioneta Roja 🚗,Para que te sientas en Camioneta Roja


In [41]:
# Удаляем исходную колонку name и переименовываем clean_name
catalog_names_clean2 = catalog_names_clean2.drop(columns=['name']).rename(columns={'name_clean': 'name'})

In [42]:
# Проверяем результат
print("\nПервые 15 записей после очистки:")
print(catalog_names_clean2.head(15))


Первые 15 записей после очистки:
    id   type                                               name
0    3  album                                    Taller Children
1   12  album                                  Wild Young Hearts
2   13  album                                      Lonesome Crow
3   17  album                                      Graffiti Soul
4   26  album                                     Blues Six Pack
5   27  album  Authorized Bootleg Live Tokyo Dome Tokyo Japan...
6   29  album                               Foot Of The Mountain
7   34  album                             Self Taught Learner EP
8   36  album                                     A Secret Place
9   39  album                                         Sunny Days
10  48  album                    THE E N D THE ENERGY NEVER DIES
11  52  album                                       Classic Rock
12  54  album                                Sides Of Blue Vol 2
13  57  album                                           

_____________________

interactions.parquet - данные о том, какие пользователи прослушали тот или иной трек:  
- user_id — идентификатор пользователя,  
- track_id — идентификатор музыкального трека,  
- track_seq — номер места трека в истории пользователя,  
- started_at — дата начала прослушивания трека.

In [43]:
interactions.head()

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21
3,0,590303,4,2022-07-22
4,0,590692,5,2022-07-22


In [44]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 5.4 GB


Проверяем данные, есть ли с ними явные проблемы.

In [45]:
interactions_clean = interactions[~interactions['track_id'].isin(track_id_deleted)].reset_index(drop=True)

In [46]:
# Убедимся, что теперь типы совпадают
print(f"Тип track_id в tracks: {tracks['track_id'].dtype}")
print(f"Тип track_id в interactions: {interactions['track_id'].dtype}")

# Проверим, что все track_id из interactions есть в tracks
missing_tracks = set(interactions['track_id']) - set(tracks['track_id'])
print(f"Отсутствующих track_id в tracks: {len(missing_tracks)}")

Тип track_id в tracks: int32
Тип track_id в interactions: int32
Отсутствующих track_id в tracks: 19023


In [47]:
interactions_clean["track_id"].nunique()

980977

In [48]:
len(interactions_clean)

222184449

In [49]:
print(f"Процент оставшихся: {len(interactions_clean)*100/len(interactions):.2f}%")

Процент оставшихся: 99.80%


In [50]:
interactions_clean[interactions_clean["track_seq"].isnull()]

,user_id,track_id,track_seq,started_at


пропусков по track_seq нет

In [51]:
interactions_clean["track_seq"].describe().apply(lambda x: f"{x:0.1f}")

count    222184449.0
mean           462.0
std            824.8
min              1.0
25%             56.0
50%            181.0
75%            505.0
max          16637.0
Name: track_seq, dtype: object

- max(track_seq) = 16637  
- min(track_seq) = 1  
- 75% `track_seq` меньше 506 
- на будущее: посчитать их за выбросы и удалить из данных

In [52]:
# оставим только взаимодействия с номерами места трека в истории пользователя меньше-равно 1000
interactions_clean = interactions_clean[interactions_clean["track_seq"] <= 1000].reset_index(drop=True)
interactions_clean.head()

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21
3,0,590303,4,2022-07-22
4,0,590692,5,2022-07-22


In [53]:
print(f"Процент оставшихся взаимодействий: {len(interactions_clean)*100/len(interactions):.2f}%")

Процент оставшихся взаимодействий: 87.81%


In [54]:
interactions_clean[interactions_clean["started_at"].isnull()]

,user_id,track_id,track_seq,started_at


пропусков по `started_at` нет

In [55]:
interactions_clean["started_at"].describe()

count                        195486833
mean     2022-08-26 18:29:16.756822016
min                2022-01-01 00:00:00
25%                2022-06-27 00:00:00
50%                2022-09-12 00:00:00
75%                2022-11-07 00:00:00
max                2022-12-31 00:00:00
Name: started_at, dtype: object

- min(started_at) = 2022-01-01  
- max(started_at) = 2022-12-31  
- все корректно, данные датированы за весь 2022 год

In [56]:
# Пример данных по взаимодействиям:
print(interactions_clean.sample(5, random_state=RANDOM_STATE)
                        .sort_values("user_id")
                        .set_index(["user_id", "track_id"])
                        .to_string())

                  track_seq started_at
user_id track_id                      
23030   62107387         62 2022-10-10
398657  38714371        473 2022-11-01
443853  2193302          36 2022-12-06
1339870 45430540        282 2022-08-31
1363369 433550           13 2022-12-23


# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

Наиболее популярные треки

Наиболее популярные жанры

Треки, которые никто не прослушал

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.